### Visible device

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [ ]:
from args import args
import pandas as pd
from prompts import summary_generation
import torch
from tqdm import tqdm
from utils.inference_helper import inference


In [ ]:
class args(args):
    model_name = ["llama3_8b", "llama3_70b", "ibm_granite_3b", "ibm_granite_8b", "ibm_granite_20b", "ibm_granite_34b"][1]
    device="cuda"
    device_auto=True
    

In [7]:
current_model = getattr(args, args.model_name)
prompt = getattr(summary_generation, args.model_name)

java_code = list(pd.read_csv(args.processed_data)['code'])

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM


if current_model['cached']:
    print("Using cached version")
    tokenizer = AutoTokenizer.from_pretrained(current_model["tokenizer"], cache_dir=args.cache_dir, device_map = 'auto') 
    model = AutoModelForCausalLM.from_pretrained(current_model["model_path"], cache_dir=args.cache_dir, device_map = 'auto') 

else:
    tokenizer = AutoTokenizer.from_pretrained(current_model["tokenizer"], device_map = 'auto')
    model = AutoModelForCausalLM.from_pretrained(current_model["model_path"], device_map = 'auto')
# device = args.device
# model.to(device)

In [18]:
generated_summaries = []

for i in tqdm(range(0, len(java_code))):
    
    current_prompt = prompt.format(java_code[i])

    with torch.no_grad():
        output = inference(tokenizer=tokenizer, model=model, prompt=current_prompt)

    generated_summaries.append(output)



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [14:09:54<00:00, 84.99s/it]


In [ ]:
summaries = pd.DataFrame()

summaries['codeXGlue_Java_code'] = java_code
summaries['generated_summaries_{}'.format(args.model_name)] = generated_summaries
summaries.to_csv('./saved_data/generated_summaries/generated_summaries_{}.csv'.format(args.model_name), index=False)